In [1]:
import numpy as np
from nltk.corpus import BracketParseCorpusReader
from nltk import *
import re


# Constants
PER_SEED = 30
NUM_SEED = 50


# Define the functions to be used

def get_next(distributions, token, order):
    dist = distributions[order-1]
    while ( len(list(dist[token].samples())) == 0):
        order -= 1
        if(order<0 or order>2):
            print("\nERROR: ",token,"HAS NO DISTRIBUTION!!!\n")
        dist = distributions[order-1]
        if (order==1):
            token = token[1]
        elif (order==2):
            token = (token[1], token[2])
            
    return dist[token].generate()


def make_ngram_sentence(distributions, seed, order=1):
    dist_b = distributions[0]
    dist_t = distributions[1]
    dist_f = distributions[2]
    i=0
    sentence = []
    sentence.append(seed)

    root = sentence[i]
    word1 = get_next(distributions, root, 1)
    word2 = get_next(distributions, root, 1)
    i += 1

    if ( dist_b[root].prob(word1) > dist_b[root].prob(word2) ):
        sentence.append(word1)
    else:
        sentence.append(word2)

    if(order == 1):
        while(sentence[i] != "" or len(sentence) <= 2 ):
            root = sentence[i]
            word1 = get_next(distributions, root, order)
            word2 = get_next(distributions, root, order)
            i += 1
        
            if ( dist_b[root].prob(word1) > dist_b[root].prob(word2) ):
                sentence.append(word1)
            else:
                sentence.append(word2)
            
        return sentence
    
    root = (sentence[i-1], sentence[i])
    word1 = get_next(distributions, root, 2)
    word2 = get_next(distributions, root, 2)
    i += 1

    if ( dist_t[root].prob(word1) > dist_t[root].prob(word2) ):
        sentence.append(word1)
    else:
        sentence.append(word2)

    if(order == 2):
        while(sentence[i] != "" or len(sentence) <= 3):
            root = (sentence[i-1], sentence[i])
            word1 = get_next(distributions, root, order)
            word2 = get_next(distributions, root, order)
            i += 1

            if ( dist_t[root].prob(word1) > dist_t[root].prob(word2) ):
                sentence.append(word1)
            else:
                sentence.append(word2)
            
        return sentence

    root = (sentence[i-2], sentence[i-1], sentence[i])
    word1 = get_next(distributions, root, 3)
    word2 = get_next(distributions, root, 3)
    i += 1

    if ( dist_f[root].prob(word1) > dist_f[root].prob(word2) ):
        sentence.append(word1)
    else:
        sentence.append(word2)

    if(order == 3):
        while(sentence[i] != "" or len(sentence) <= 4):
            root = (sentence[i-2], sentence[i-1], sentence[i])
            word1 = get_next(distributions, root, order)
            word2 = get_next(distributions, root, order)
            i += 1

            if ( dist_f[root].prob(word1) > dist_f[root].prob(word2) ):
                sentence.append(word1)
            else:
                sentence.append(word2)
            
        return sentence

    
    

# Edit this one


def get_ngram_probability(distributions, ngram_input, sentence, order=1):
    dist_b = distributions[0]
    dist_t = distributions[1]
    dist_f = distributions[2]

    prob = np.log(ngram_input.count(sentence[0])/len(ngram_input))
    prob += np.log( dist_b[sentence[0]].prob(sentence[1]) )

    if(order == 1):
        for i in range(2,len(sentence)):
            condition = sentence[i-1]
            prob += np.log(dist_b[condition].prob(sentence[i]))
            
        return prob/len(sentence)
    
    condition = (sentence[0], sentence[1])
    prob += np.log( dist_t[condition].prob(sentence[2]) )

    if(order == 2):
        for i in range(3,len(sentence)):
            condition = (sentence[i-2], sentence[i-1])
            prob += np.log(dist_t[condition].prob(sentence[i]))
            
        return prob/len(sentence)

    condition = (sentence[0], sentence[1], sentence[2])
    prob += np.log( dist_f[condition].prob(sentence[3]) )

    if(order == 3):
        for i in range(4,len(sentence)):
            condition = (sentence[i-3], sentence[i-2], sentence[i-1])
            prob += np.log(dist_f[condition].prob(sentence[i]))
            
        return prob/len(sentence)


In [2]:
# Import and parse the corpus

corpus_root = 'C:/MSCS-Fall17(courses)/sem2/NLP/Project/corpus_clean/'
corpus = BracketParseCorpusReader(corpus_root, ".*")

tagged_sentences = corpus.tagged_sents()
ngram_input = []
pos_input = []
legal_tags = ["EOS","$","#", "GW", "CC", "CD", "DT", "EX", "FW", "IN", "JJ","JJR","JJS","LS","MD",
             "NN","NNS","NNP",'NNPS','PDT','POS','PRP','PRP$','RB','RBR','RBS','RP','TO', "UH",'VB',
             'VBD',"VBG","VBN","VBP","VBZ","WDT","WP","WP$","WRB", "\"", "\'", ",", ".", "AFX"]

single_letter_words = ["a", "i", ",", ".", "!", "?", "\'", "\"", ":", ';', '0', '1', '2', "3", '4',
                       '5', "6", '7', '8', "9", "=", "&", "#", '/', '>', "$", '<', '+', '%',]

# tags_removed = ["-NONE-","SYM", "CODE", "ADD", "HYPH","-LSB-", "-RSB-",":", "NFP", "XX", "-LRB-", "-RRB-"]

#  Remove -NONE- and  SYM tags from the training data and create a list of tokens and a list of tags.
for sentence in tagged_sentences:
    for token in sentence:
        word = token[0].lower()
        tag = token[1]
        
        if(tag == "NP"):
            tag = "NNS"

        if not tag in legal_tags:
            del token
            continue
        
        if len(word) == 1:
            if not word in single_letter_words:
                del token
                continue
        
        if (word[0:5] == "rsquo"):
            word = "\'" + word[5:]

        ngram_input.append(word)
        pos_input.append(tag)

    ngram_input.append("")
    pos_input.append("EOS")

unique_alphas = []

for string in list(set(ngram_input)):
    if string[0:1].isalpha():
        unique_alphas.append(string)

print("There are",len(ngram_input),"tokens in the corpus.")
print("There are",len(unique_alphas),"unique tokens that start with a letter.")


tag_set = set(pos_input)
print("There are",len(tag_set),"unique tags in the corpus.")

There are 517943 tokens in the corpus.
There are 25116 unique tokens that start with a letter.
There are 44 unique tags in the corpus.


In [3]:
print("http:$$$$demo.businesslayers.com" in ngram_input)
print("ADD" in pos_input)

False
False


In [4]:
# Create bigram and trigram lists
bgram = list(ngrams(ngram_input,2))
tgram = list(ngrams(ngram_input,3))
fgram = list(ngrams(ngram_input,4))

pos_bgram = list(ngrams(pos_input,2))


# Create conditional frequency distributions
cfd_b = ConditionalFreqDist(bgram)

cfd_t = ConditionalFreqDist()
for trigram in tgram:
    condition = (trigram[0], trigram[1])
    cfd_t[condition][trigram[2]] += 1

cfd_f = ConditionalFreqDist()
for fourgram in fgram:
    condition = (fourgram[0], fourgram[1], fourgram[2])
    cfd_f[condition][fourgram[3]] += 1

cfd_pos = ConditionalFreqDist(pos_bgram)

cfd_t2w = ConditionalFreqDist()
for tag, word in zip(pos_input, ngram_input):
    cfd_t2w[tag][word] += 1

cfd_w2t = ConditionalFreqDist()
for tag, word in zip(pos_input, ngram_input):
    cfd_w2t[word][tag] += 1

# Create conditional probability distributions
cpd_b = ConditionalProbDist(cfd_b, MLEProbDist)
cpd_t = ConditionalProbDist(cfd_t, MLEProbDist)
cpd_f = ConditionalProbDist(cfd_f, MLEProbDist)

cpd_pos = ConditionalProbDist(cfd_pos, MLEProbDist)
cpd_t2w = ConditionalProbDist(cfd_t2w, MLEProbDist)
cpd_w2t = ConditionalProbDist(cfd_w2t, MLEProbDist)


# Consolidate the ngram probability distributions into a single object
distributions = [cpd_b, cpd_t, cpd_f]



print("There are",len(bgram),"bigrams.")
print("There are",len(tgram),"trigrams.")
print("There are",len(fgram),"fourgrams.")

There are 517942 bigrams.
There are 517941 trigrams.
There are 517940 fourgrams.


In [5]:
seed_list = np.random.choice(unique_alphas,NUM_SEED)

print(seed_list,"\n\n")

#Sentences using Bigram
for i in range(NUM_SEED):

    sentences = []
    full_sentences = []
    prob = []

    for j in range(PER_SEED):
        sentences.append(make_ngram_sentence(distributions, "", 1))
        prob.append(get_ngram_probability(distributions, ngram_input, sentences[j], 1))
        full_sentences.append(" ".join(sentences[j]))

    best_index = np.argmax(prob)
    print(full_sentences[best_index],"\n",prob[best_index],"\n")

['intervenes' 'pinch' 'cardmember' 'fueled' 'downs' 'guatemala' 'nfs'
 'relatively' 'transylvania' 'symposium' 'us$' 'transcend' 'unicode' 'mtas'
 'non-blurry' 'jacob' 'occurs' 'booming' 'fall' 'worn' 'julianne'
 'reconsider' 'definite' 'undulates' 'germany' 'basso' 'tavern' 'resurrect'
 'clad' 'cob' 'candy' 'grows' 'othello' 'concessions' 'esteemed' 'na'
 'reduction' 'harbor' 'flareup' 'mime' 'cleansed' 'homosexual' 'subscribe'
 'meaness' 'rhythmic' 'unintended' 'theater' 'pelletier' 'briskly'
 'antihomosexual'] 


intervenes between the one of the energy , but i 'll have the effects of the stlp strawman document .  
 -3.37340128464 

pinch hitter .  
 -3.30343402845 

cardmember we can make a shaved head motive of the ectodermal ridge pre-aer at school board meeting .  
 -3.19123081724 

fueled by the local variations in the number of the femme fatale .  
 -3.23071393331 

downs  the kursk was because i have been a bed load channel will receive an investment in east coast pinot noir 

In [6]:
#Sentences using Trigram
for i in range(NUM_SEED):

    sentences = []
    full_sentences = []
    prob = []

    for j in range(PER_SEED):
        sentences.append(make_ngram_sentence(distributions, "", 2))
        prob.append(get_ngram_probability(distributions, ngram_input, sentences[j], 2))
        full_sentences.append(" ".join(sentences[j]))

    best_index = np.argmax(prob)
    print(full_sentences[best_index],"\n",prob[best_index],"\n")

intervenes between e♭ and d♭ in measure 293 , preceded by a♭ major in measure 285 , suggests that seemingly random micro-level behaviour .  
 -0.971892239229 

pinch hitter .  
 -3.28940511914 

cardmember we re pleased to welcome you aboard 747 flight 602 from new york was still essentially a communist dictator .  
 -1.07407781248 

fueled by the postal service a clear pattern also emerges that reveals micro-level organising shaping macro activity .  
 -1.21580312081 

downs  a potentially more interesting than the average american and european publications far better than reality .  
 -1.7549021895 

guatemala over 450 villages were destroyed , 200,000 mayans were killed and 23 others according to guncite , are almost a direct function of the best fit of equatorial ligands of scd39 bound vo2+ epr spectrum of bound vo2+ epr spectrum of bound vo2+ amppnp .  
 -1.29500656897 

nfs for the appreciation of color in the non-neural cell types , including cockamamy or cockamamie ' mixed-up ,

subscribe to that , i have to put the plant physiologic underpinnings of why some companies rise from difficulty to become vigilant in safe-guarding the lives of the senate gifts rule 35 and , in fact , upstream deposition will reduce downstream sediment loads may be some knowledge that i 'm telling you , and that 's what i 've had some thoughts about what might have been sold was shaking in a flash of useful arts and science  
 -1.55356630395 

meaness most of the universe just falls apart when its string is released but draws details from other buildings of the bible .  
 -1.56237103228 

rhythmic pattern  we had this guy was asked by people sitting on a split decision uh go with you and your loved ones who get there early if you would like to see the spirits undoubtably floating around her hands dance in mid-air , looking at the school board was doing during this period , but he was on his shoulder , closing the box , page 14 , vice president gore , governor , both of them .  
 -1.5

In [7]:
#Sentences using four-gram
for i in range(NUM_SEED):

    sentences = []
    full_sentences = []
    prob = []

    for j in range(PER_SEED):
        sentences.append(make_ngram_sentence(distributions, "", 3))
        prob.append(get_ngram_probability(distributions, ngram_input, sentences[j], 3))
        full_sentences.append(" ".join(sentences[j]))

    best_index = np.argmax(prob)
    print(full_sentences[best_index],"\n",prob[best_index],"\n")

intervenes between e♭ and d♭ in measure 195 ? another instance of the fugato section , not reproduced in its entirety but based on the ratio of atpase adpase activity are present on the cell surface e-ntpdase1 with a ratio of 10 and e-ntpdase3 with a ratio of 2 2 , the entire development section is divided into eighteen chapters varying in length , each dealing with a fact witness , but these breaches stretched from the negev to the galilee .  
 -0.347323225826 

pinch hitter .  " if you look around your community you 'll find all kinds of clothing , the baggy pegged pants and jackets with padded shoulders , that later became known as the " dominant " relationship in this movement .  
 -0.710627557123 

cardmember we re pleased to send you the nwf tote bag with your welcome package when we receive your commitment , we can avoid years of litigation , business uncertainty and costs , and increase collagen degradation .  
 -0.535289896712 

fueled by low interest rates , widely available 

 -0.33245033071 

resurrect happiness .  in 1419 , filippo brunelleschi began the foundling hospital in florence , whose delicate arcade of corinthian columns surmounted by pedimented windows is generally considered the first building of the national beer wholesalers association and and appears to be important to rebuild that coalition to keep the parties at the table , her lips soft and firm and sticking just enough that she gave the feeling you were really kissing , not just for kids .  
 -0.411411254036 

clad in titanium , previously used chiefly for building aircraft .  
 -1.09646837305 

cob if at all , the sort of romance that sustains the hopeless billions who regularly watch a wandering star called love .  
 -0.817979632205 

candy .  end of flame , if you will , at the end of the workshop , a woman walked up to emil 's home while i watched the screen switch to yahoo , and was the one who does n't have much control over jobs .  
 -0.729068565369 

grows is not to deceive the v

In [8]:
def get_next_tag(pos_dist, tag):
    return pos_dist[tag].generate()

def get_next_word(t2w_dist, tag):
    return t2w_dist[tag].generate()

def get_pos_probability(pos_dist, t2w_dist, w2t_dist, pos_input, tag_sentence):
    prob = np.log(pos_input.count(tag_sentence[0][0])/len(pos_input))
    
    for i in range(1,len(tag_sentence)):
        prob += np.log(dist[tag_sentence[i-1]].prob(sentence[i]))

    return prob/len(sentence)


def make_pos_sentence(pos_dist, t2w_dist, w2t_dist, seed):
    i=0
    tag = w2t_dist[seed].generate()

    sentence = []
    sentence.append("")

    tags = []
    tags.append("EOS")

    while(sentence[i] != "" or len(sentence) <= 2 ):
        tags.append(get_next_tag(pos_dist, tags[i]))
        sentence.append(get_next_word(t2w_dist, tags[i]))
        i += 1
        
    return sentence

for i in range(len(seed_list)):
    pos_sent_list = make_pos_sentence(cpd_pos, cpd_t2w, cpd_w2t,seed_list[i])
    pos_sent = " ".join(pos_sent_list)
    print(str(pos_sent)+"\n")
    


  we was i believed , and revenues , with something . 

  standing the meeting to get legally somehow in both response of next standard with same emissions that were the " 

  is why malleable native one territory . 

  we was from really of the legitimacy between no writing . 

  alongside the disorders what has whole next in regulations buildings that is just say to world 

  or is of your pagoda to some perfect , and he do to maintenance through reports and to have my grassy to rebuild message 's religion might empire institute , with schlemmer florida because a horror of the left . 

  china hittite arabs were i was for a subtle matter . 

  the system and simply called one forms for an who is a european and a sound harms am concerning in a media . 

  i could tell , camera above the states " you bad is then of you think theme men it , told as warren bubbled because the stubborn food here = end working on sure name after share if a tax to go clips , you leave buttered new one behin